In [40]:
import tensorflow as tf
from tensorflow.keras import models
from keras import layers
from keras.applications import Xception
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import multi_gpu_model
from keras.optimizers import SGD, RMSprop, Adam
from keras import metrics
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow.python'

In [3]:
# 데이터 불러오기
train_features = pd.read_csv('./original/train_features.csv')
train_target = pd.read_csv('./original/train_target.csv', index_col = 'id')
test_features = pd.read_csv('./original/test_features.csv')

In [4]:
print(f'train_features {train_features.shape}')
print(f'train_target {train_target.shape}')
print(f'test_features {test_features.shape}')

train_features (1050000, 6)
train_target (2800, 4)
test_features (262500, 6)


In [26]:
display(train_features.tail())
display(train_target.head())

,id,Time,S1,S2,S3,S4
1049995,2799,0.001480,-89256.32,-164614.6,48186.29,-147910.20
1049996,2799,0.001484,-29233.44,-179404.2,109534.40,-111591.10
1049997,2799,0.001488,37147.25,-190011.7,178781.10,-86836.78
1049998,2799,0.001492,98278.50,-190771.4,245726.70,-75660.00
1049999,2799,0.001496,151683.00,-177267.3,307932.80,-72835.82


,X,Y,M,V
id,,,,
0,0.0,-400.0,50.0,0.4
1,400.0,0.0,100.0,1.0
2,-300.0,-200.0,25.0,0.4
3,200.0,-100.0,150.0,0.4
4,-300.0,-100.0,150.0,0.4


### 데이터에 대한 설명

In [28]:
print('''train features
id = 충돌체 고유 아이디
time = 관측시간(second)
s1 = S1 센서에서 측정된 가속도 (mm/s^2)
s2 = S2 센서에서 측정된 가속도 (mm/s^2)
s3 = S3 센서에서 측정된 가속도 (mm/s^2)
s4 = S4 센서에서 측정된 가속도 (mm/s^2)

train_target
id = 충돌체 고유 아이디
X, Y = 충격하중이 가해진 X, Y 좌표 (mm)
M = 충돌체의 질량 (g)
V = 충돌체의 충돌 속도 (m/s)
''')

train features
id = 충돌체 고유 아이디
time = 관측시간(second)
s1 = S1 센서에서 측정된 가속도 (mm/s^2)
s2 = S2 센서에서 측정된 가속도 (mm/s^2)
s3 = S3 센서에서 측정된 가속도 (mm/s^2)
s4 = S4 센서에서 측정된 가속도 (mm/s^2)

train_target
id = 충돌체 고유 아이디
X, Y = 충격하중이 가해진 X, Y 좌표 (mm)
M = 충돌체의 질량 (g)
V = 충돌체의 충돌 속도 (m/s)



In [31]:
print('''id 각각은 375 row를 가짐
id는 0 ~ 2779''')

id 각각은 375 row를 가짐
id는 0 ~ 2779


## 우선은 id=0인 값들로 먼저 예측 모델을 짜보자

In [23]:
df_0 = train_features[train_features['id'] == 0]
display(df_0)

,id,Time,S1,S2,S3,S4
0,0,0.000000,0.00,0.00,0.00,0.000
1,0,0.000004,0.00,0.00,0.00,0.000
2,0,0.000008,0.00,0.00,0.00,0.000
3,0,0.000012,0.00,0.00,0.00,0.000
4,0,0.000016,0.00,0.00,0.00,0.000
...,...,...,...,...,...,...
370,0,0.001480,-64168.90,-64168.90,52279.59,106792.600
371,0,0.001484,-64236.79,-64236.79,16518.64,58248.420
372,0,0.001488,-63755.95,-63755.95,-25270.30,3015.649
373,0,0.001492,-63020.44,-63020.44,-65904.66,-49795.140
